In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
!pip install mediapipe==0.10.14


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.7/35.7 MB 45.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 15.2 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.5
    Uninstalling protobuf-5.29.5:
      Successfully uninstalled protobuf-5.29.5
  Attempting uninstall: mediapipe
    Found existing installation: mediapipe 0.10.31
    Uninstalling mediapipe-0.10.31:
      Successfully uninstalled mediapipe-0.10.31
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-adk 1.22.1 requires google-cloud-bigquery-storage>=2.0.0, which is not installed.
bigframes 2.26.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
a2a-sdk 0.3.22 requires protobuf>=5.29.5, but you have protobuf 4.25.8 which is incompatible.
opentelemetry-proto 1.37.0 requires protob

In [6]:
import mediapipe
import os

print("1. Location of imported mediapipe:")
if hasattr(mediapipe, '__file__'):
    print(mediapipe.__file__)
else:
    print("mediapipe has no __file__ attribute (This is a bad sign).")

print("\n2. Files in current directory:")
print(os.listdir())

1. Location of imported mediapipe:
/usr/local/lib/python3.12/dist-packages/mediapipe/__init__.py

2. Files in current directory:
['.virtual_documents']


In [7]:
import mediapipe as mp

# 1. Manually import the solutions module
import mediapipe.python.solutions as solutions

# 2. Assign it back to mp (optional, but keeps your old code working)
mp.solutions = solutions

# 3. Now try your original setup
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(min_detection_confidence=0.6, min_tracking_confidence=0.6)

print("Success! Pose module loaded.")

Success! Pose module loaded.


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1768717856.619470     154 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1768717856.662489     154 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


In [8]:

import cv2
import mediapipe as mp


import math

VIDEO_PATH = '/kaggle/input/newtoetap/Toe Taps.mp4'
OUTPUT_VIDEO_PATH = 'annotated_toe_taps_final.mp4'

TOUCH_THRESHOLD = 35
TOUCH_COOLDOWN_FRAMES = 8
VELOCITY_FRAME_DELTA = 5

LOWER_COLOR_BOUND = np.array([0, 0, 150])
UPPER_COLOR_BOUND = np.array([180, 60, 255])

# Corrected import for mp_pose
mp_pose = mp.solutions.pose # Uncommented and using mp.solutions.pose
pose = mp_pose.Pose(min_detection_confidence=0.6, min_tracking_confidence=0.6) # Using mp_pose.Pose

def find_ball_optimized(frame):
    hsv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(hsv_frame, LOWER_COLOR_BOUND, UPPER_COLOR_BOUND)
    mask = cv2.erode(mask, None, iterations=2)
    mask = cv2.dilate(mask, None, iterations=2)
    contours, _ = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    best_candidate = None
    best_score = 0
    if len(contours) > 0:
        for c in contours:
            area = cv2.contourArea(c)
            if area < 100 or area > 5000:
                continue
            perimeter = cv2.arcLength(c, True)
            if perimeter == 0:
                continue
            circularity = 4 * np.pi * (area / (perimeter * perimeter))
            if 0.7 < circularity < 1.2:
                if circularity > best_score:
                    best_score = circularity
                    best_candidate = c
    if best_candidate is not None:
        ((x, y), radius) = cv2.minEnclosingCircle(best_candidate)
        M = cv2.moments(best_candidate)
        center = (int(M["m10"] / M["m00"]), int(M["m01"] / M["m00"]))
        bbox = (int(x - radius), int(y - radius), int(radius * 2), int(radius * 2))
        return center, bbox
    return None, None

def run_video_analysis(video_path):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error: Could not open video file at '{video_path}'. Please check the path.")
        return
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS)) if int(cap.get(cv2.CAP_PROP_FPS)) > 0 else 30
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(OUTPUT_VIDEO_PATH, fourcc, fps, (width, height))
    print("Starting video processing...")
    touch_count_left, touch_count_right = 0, 0
    ball_rotation, player_velocity = "N/A", 0.0
    touch_cooldown = 0
    player_positions, prev_gray_frame = [], None
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        if prev_gray_frame is None:
            prev_gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        ball_center, ball_bbox = find_ball_optimized(frame)
        if ball_center:
            cv2.rectangle(frame, (ball_bbox[0], ball_bbox[1]),
                          (ball_bbox[0] + ball_bbox[2], ball_bbox[1] + ball_bbox[3]),
                          (0, 255, 0), 2)
        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(image_rgb)
        if results.pose_landmarks:
            landmarks = results.pose_landmarks.landmark
            left_toe = (int(landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX].x * width), int(landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX].y * height)) # Using mp_pose.PoseLandmark
            right_toe = (int(landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX].x * width), int(landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX].y * height)) # Using mp_pose.PoseLandmark
            left_hip = (int(landmarks[mp_pose.PoseLandmark.LEFT_HIP].x * width), int(landmarks[mp_pose.PoseLandmark.LEFT_HIP].y * height)) # Using mp_pose.PoseLandmark
            right_hip = (int(landmarks[mp_pose.PoseLandmark.RIGHT_HIP].x * width), int(landmarks[mp_pose.PoseLandmark.RIGHT_HIP].y * height)) # Using mp_pose.PoseLandmark
            player_center = ((left_hip[0] + right_hip[0]) / 2, (left_hip[1] + right_hip[1]) / 2)
            player_positions.append(player_center)
            if len(player_positions) > VELOCITY_FRAME_DELTA:
                player_positions.pop(0)
            if ball_center and touch_cooldown == 0:
                dist_left = math.hypot(ball_center[0] - left_toe[0], ball_center[1] - left_toe[1])
                dist_right = math.hypot(ball_center[0] - right_toe[0], ball_center[1] - right_toe[1])
                if min(dist_left, dist_right) < TOUCH_THRESHOLD:
                    touch_cooldown = TOUCH_COOLDOWN_FRAMES
                    if len(player_positions) == VELOCITY_FRAME_DELTA:
                        past_pos = player_positions[0]
                        pixel_dist = math.hypot(player_center[0] - past_pos[0], player_center[1] - past_pos[1])
                        time_elapsed = VELOCITY_FRAME_DELTA / fps
                        player_velocity = pixel_dist / time_elapsed if time_elapsed > 0 else 0
                    if dist_left < dist_right:
                        touch_count_left += 1
                    else:
                        touch_count_right += 1
        if ball_bbox:
            gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            ball_roi = gray_frame[ball_bbox[1]:ball_bbox[1]+ball_bbox[3], ball_bbox[0]:ball_bbox[0]+ball_bbox[2]]
            prev_ball_roi = prev_gray_frame[ball_bbox[1]:ball_bbox[1]+ball_bbox[3], ball_bbox[0]:ball_bbox[0]+ball_bbox[2]]
            if ball_roi.size > 0 and prev_ball_roi.size > 0 and ball_roi.shape == prev_ball_roi.shape:
                flow = cv2.calcOpticalFlowFarneback(prev_ball_roi, ball_roi, None, 0.5, 3, 15, 3, 5, 1.2, 0)
                avg_flow_x = np.mean(flow[..., 0])
                if avg_flow_x > 1.0: ball_rotation = "Forward"
                elif avg_flow_x < -1.0: ball_rotation = "Backward"
            prev_gray_frame = gray_frame
        if touch_cooldown > 0:
            touch_cooldown -= 1
        cv2.rectangle(frame, (0, height-100), (width, height), (0, 0, 0), -1)
        cv2.putText(frame, f"Right Leg Taps: {touch_count_right}", (20, height - 65), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
        cv2.putText(frame, f"Left Leg Taps: {touch_count_left}", (20, height - 25), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
        cv2.putText(frame, f"Rotation: {ball_rotation}", (width - 450, height - 65), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
        cv2.putText(frame, f"Velocity: {player_velocity:.2f} px/sec", (width - 450, height - 25), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
        out.write(frame)
    cap.release()
    out.release()
    print(f"✅ Processing complete! Annotated video saved to '{OUTPUT_VIDEO_PATH}'")
run_video_analysis(VIDEO_PATH)


W0000 00:00:1768717870.126441     159 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1768717870.166012     159 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


Starting video processing...


/usr/local/lib/python3.12/dist-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


✅ Processing complete! Annotated video saved to 'annotated_toe_taps_final.mp4'
